# 1. Import

In [ ]:
%load_ext autoreload
%autoreload 2

import cfg
from src import data_loader, models, trainer, evaluate
from torchinfo import summary
import torch.nn as nn
import torch.optim as optim

trainer.set_seed(cfg.RANDOM_STATE)


# 2. Data loading

In [ ]:
if cfg.IMBALANCED_MODE == True: 
    X_train, Y_train, X_val, Y_val, X_test, Y_test = data_loader.data_import(cfg.OVERLAPPING_RATIO,cfg.BASE_PATH,cfg.SAMPLE_LENGTH, cfg.PREPROCESSING)
    X_train_imbalanced, Y_train_imbalanced = data_loader.create_imbalanced_data(X_train, Y_train, cfg.IMBALANCED_RATIO, cfg.NUM_CLASSES)
    X_train_normalized, X_val_normalized, X_test_normalized = data_loader.normalize_data(X_train_imbalanced, X_val, X_test)
    X_train_smoted, Y_train_smoted = data_loader.apply_smote(X_train_normalized, Y_train_imbalanced, cfg.RANDOM_STATE, cfg.SAMPLE_LENGTH)
    train_loader, val_loader, test_loader = data_loader.create_dataloaders(X_train_smoted, Y_train_smoted,
                                                                            X_val_normalized, Y_val,
                                                                            X_test_normalized, Y_test,
                                                                            cfg.SAMPLE_LENGTH, cfg.CNN1D_INPUT, cfg.BATCH_SIZE, cfg.RANDOM_STATE)
else:
    X_train, Y_train, X_val, Y_val, X_test, Y_test = data_loader.data_import(cfg.OVERLAPPING_RATIO,cfg.BASE_PATH,cfg.SAMPLE_LENGTH,cfg.PREPROCESSING)
    X_train_normalized, X_val_normalized, X_test_normalized = data_loader.normalize_data(X_train, X_val, X_test)
    train_loader, val_loader, test_loader = data_loader.create_dataloaders(X_train_normalized, Y_train,
                                                                            X_val_normalized, Y_val,
                                                                            X_test_normalized, Y_test,
                                                                            cfg.SAMPLE_LENGTH, cfg.CNN1D_INPUT, cfg.BATCH_SIZE, cfg.RANDOM_STATE)

# 3. Model defining

In [ ]:
if cfg.MODEL_TYPE == 'student':
  print("Using StudentModel")
  model_test = models.StudentModel_Improved(cfg.NUM_CLASSES)
if cfg.MODEL_TYPE == 'teacher':
  print("Using TeacherModel")
  model_test = models.TeacherModel(cfg.NUM_CLASSES)

if cfg.MODEL_TYPE == '1D':
  print("Using 1DCNNModel")
  model_test = models.Basic1DCNN(1,cfg.NUM_CLASSES)

if cfg.MODEL_TYPE == '1D-Basic':
  print("Using 1DCNNModel")
  model_test = models.Basic1DCNN(1, cfg.NUM_CLASSES)
  
# summary(model_test, input_size=(cfg.BATCH_SIZE, 1,cfg.LENGTH))
size = int(cfg.SAMPLE_LENGTH**(1/2))
summary(model_test, input_size=(cfg.BATCH_SIZE, 1, size, size))


# 4. Training

In [ ]:
print(f'Using device: {cfg.DEVICE}')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_test.parameters(), lr= cfg.LEARNING_RATE)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max= cfg.NUM_EPOCHS, eta_min=cfg.LEARNING_RATE *1e-2)
trained_model, history = trainer.train_model(model_test, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=cfg.NUM_EPOCHS, device=cfg.DEVICE)

# 5. Evaluating

In [ ]:
print("Confusion Matrix")
evaluate.plot_confusion_matrix(trained_model, test_loader, 'cpu', cfg.CLASS_NAMES)

print("\nt-SNE")
evaluate.plot_tsne(trained_model, test_loader, 'cpu', cfg.CLASS_NAMES, n_iter=1000, perplexity=30) # n_iter và perplexity có thể cần điều chỉnh

print("\nHistory")
evaluate.plot_history(history)

In [ ]:
print("Confusion Matrix")
evaluate.plot_confusion_matrix(trained_model, val_loader, 'cpu', cfg.CLASS_NAMES)

print("\nt-SNE")
evaluate.plot_tsne(trained_model, val_loader, 'cpu', cfg.CLASS_NAMES, n_iter=1000, perplexity=30) # n_iter và perplexity có thể cần điều chỉnh

In [ ]:
print("Confusion Matrix")
evaluate.plot_confusion_matrix(trained_model, train_loader, 'cpu', cfg.CLASS_NAMES)

print("\nt-SNE")
evaluate.plot_tsne(trained_model, train_loader, 'cpu', cfg.CLASS_NAMES, n_iter=1000, perplexity=30) # n_iter và perplexity có thể cần điều chỉnh